Exponential smoothing methods are renowned for being robust to small structural changes in the time series. The Holt-Winters method, which extends exponential smoothing to capture seasonality, is particularly useful for data with a clear seasonal pattern.

In [8]:
from fbprophet import Prophet
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Load historical data
data = yf.download('AAPL', start='2015-01-01', end='2023-01-01')
df = data.reset_index()[['Date', 'Close']]
df.rename(columns={'Date': 'ds', 'Close': 'y'}, inplace=True)

# Fit the Prophet model without considering holidays
model = Prophet(daily_seasonality=True)  # Ensure daily seasonality is handled, if appropriate
model.fit(df)

# Make a future dataframe for predictions; here, let's predict for the next 365 days
future = model.make_future_dataframe(periods=365, freq='D')

# Generate forecasts
forecast = model.predict(future)

# Plot the forecast
fig1 = model.plot(forecast)

# Assuming you want to evaluate using the last year of actual data for now
# Note: Actual comparison should use out-of-sample data, adjust as needed
actual = df.set_index('ds')['y'][-365:]  # Last 365 days from the loaded data
predicted_dates = forecast.set_index('ds')['yhat'][-365:]  # Corresponding predicted data

# Ensure alignment by comparing the same dates
actual_aligned, predicted_aligned = actual.align(predicted_dates, join='inner')

# Define the function to calculate metrics
def calculate_metrics(actual, predicted):
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    mae = mean_absolute_error(actual, predicted)
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    return {'RMSE': rmse, 'MAE': mae, 'MAPE': mape}

# Calculate evaluation metrics
metrics = calculate_metrics(actual_aligned, predicted_aligned)
print("Evaluation Metrics:", metrics)


TypeError: This is a python-holidays entity loader class. For entity inheritance purposes please import a class you want to derive from directly: e.g., `from holidays.countries import Entity` or `from holidays.financial import Entity`.